# Yes24 카테고리별 베스트셀러 크롤링
- 국내도서 하위메뉴 한정. 베스트셀러
    - 소설/시/희곡
    - 에세이
    - 인문
    - 역사
    - 예술
    - 종교
    - 사회 정치
    - 자연과학
    - 경제 경영
    - 자기계발
- 각 카테고리당 (15 * 40 = 600권씩 수집)
    - 총 10 * (15 * 40) = 6000개의 데이터

In [1]:
!pip install requests

In [2]:
!pip install beautifulsoup4

In [3]:
!pip install krwordrank

In [4]:
!pip install JPype1-1.2.0-cp37-cp37m-win_amd64.whl --user

Processing d:\jupyter\book-recommendation\jpype1-1.2.0-cp37-cp37m-win_amd64.whl
JPype1 is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [ ]:
!pip install konlpy

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
from krwordrank.word import summarize_with_keywords
from konlpy.tag import Okt

- ~~이미지 링크 넣기~~
    - "http://image.yes24.com/goods/64337761/L" 형식
- 별점 있고 8.7 미만인 것 삭제
- feature 컬럼으로 모으기
- 시리즈, 전집 등은...
    - 시리즈, 전집 등으로 검색해서 포함하는 row 삭제
    - 숫자로 끝나거나, 권으로 끝나는 것
    - 괄호?
- 중복 삭제 (책 제목 기준으로)
- 인물은 없애도 될듯!

In [2]:
import os
path = './crawled_data/pkl/이미지추가/'
file_list = os.listdir(path)
files = [file for file in file_list if file.endswith('.pkl')]
files

['book_info_경제&경영.pkl',
 'book_info_사회&정치.pkl',
 'book_info_소설&시&희곡.pkl',
 'book_info_에세이.pkl',
 'book_info_역사.pkl',
 'book_info_예술.pkl',
 'book_info_인문.pkl',
 'book_info_자기계발.pkl',
 'book_info_자연과학.pkl']

## 전처리
1. title, author 없는 것 제거
2. rating 컬럼을 숫자형 컬럼으로 바꿔주기 (pd.to_numeric) 
3. 별점 0 이상 8.5 미만 제거
4. 책 제목이 같은 데이터 중복 제거 & 시리즈, 전집

In [33]:
book_info_0 = pd.read_pickle(path+files[0])
book_info_0

,title,author,rating,category,tag,keyword,image
0,매매의 기술,박병창,9.5,주식 투자 재테크 재테크일반 경제 경영 증권,주식투자입문 주식투자,,http://image.yes24.com/goods/101666872/L
1,부의 시나리오,오건영,,경제 경영 투자 재테크 재테크일반,투자트렌드 포트폴리오 삼프로TV,금융시장 투자 경제 오건,http://image.yes24.com/goods/101874075/L
2,라오어의 미국주식 무한매수법,라오어,10.0,주식 투자 재테크 경제 경영 증권,미국주식,,http://image.yes24.com/goods/101875887/L
3,부자 아빠 가난한 아빠 20주년 특별 기념판,로버트 기요사키 안진환,9.4,경제 경영 투자 재테크 재테크일반,돈모으고싶을때 금융문맹탈출 가난이싫어 부자되는법,아빠 금융 부자 세계 투자 아버 대한 가지 사람 내용,http://image.yes24.com/goods/58774995/L
4,나는 당신이 주식 공부를 시작했으면 좋겠습니다,이효석,10.0,주식 투자 재테크 경제 경영 증권,삼프로TV 주식투자입문 주식투자,투자 생존 보이지 주식 저자 미래 전문,http://image.yes24.com/goods/101619905/L
...,...,...,...,...,...,...,...
595,경제용어도감,하나오카 사치코 하마바타케 가노우 송서휘,10.0,경제 경영 경제이야기 경제상식 경제,효율적인공부 상식,경제 이해 하나 직장 일러 용어 문장 개념 모든,http://image.yes24.com/goods/73287855/L
596,OKR 전설적인 벤처투자자가 구글에 전해준 성공 방식,존 도어 박세연 이길상,9.3,경제 경영 경영 경영혁신 경영전략,빌게이츠추천 실리콘밸리 글로벌기업,,http://image.yes24.com/goods/70981291/L
597,취업 끝판왕 옴스에게 배우는 스펙을 뛰어넘는 자소서,옴스,9.8,비즈니스맨 자기계발 CEO 유망직업 수험서 자격증 취업 상식 적성검사 경제 경영 면접,일생의기술,직무 소서 경험 자신,http://image.yes24.com/goods/69338137/L
598,나는 오를 땅만 산다,김종율,9.2,투자 경매 부동산 경제 경영 재테크,돈좀벌어보자 부자되는법 가난이싫어 안정적투자,토지 사례 가치 투자,http://image.yes24.com/goods/67096736/L


In [8]:
book_info_0[book_info_0['author'] == '']

,title,author,rating,category,tag,keyword,image
478,알기쉬운 경제지표해설,,10.0,경제 경영 정부간행물,,,http://image.yes24.com/goods/68377557/L


In [4]:
book_info_0['rating'] = pd.to_numeric(book_info_0['rating'])
book_info_0

,title,author,rating,category,tag,keyword,image
0,매매의 기술,박병창,9.5,주식 투자 재테크 재테크일반 경제 경영 증권,주식투자입문 주식투자,,http://image.yes24.com/goods/101666872/L
1,부의 시나리오,오건영,NaN,경제 경영 투자 재테크 재테크일반,투자트렌드 포트폴리오 삼프로TV,금융시장 투자 경제 오건,http://image.yes24.com/goods/101874075/L
2,라오어의 미국주식 무한매수법,라오어,10.0,주식 투자 재테크 경제 경영 증권,미국주식,,http://image.yes24.com/goods/101875887/L
3,부자 아빠 가난한 아빠 20주년 특별 기념판,로버트 기요사키 안진환,9.4,경제 경영 투자 재테크 재테크일반,돈모으고싶을때 금융문맹탈출 가난이싫어 부자되는법,아빠 금융 부자 세계 투자 아버 대한 가지 사람 내용,http://image.yes24.com/goods/58774995/L
4,나는 당신이 주식 공부를 시작했으면 좋겠습니다,이효석,10.0,주식 투자 재테크 경제 경영 증권,삼프로TV 주식투자입문 주식투자,투자 생존 보이지 주식 저자 미래 전문,http://image.yes24.com/goods/101619905/L
...,...,...,...,...,...,...,...
595,경제용어도감,하나오카 사치코 하마바타케 가노우 송서휘,10.0,경제 경영 경제이야기 경제상식 경제,효율적인공부 상식,경제 이해 하나 직장 일러 용어 문장 개념 모든,http://image.yes24.com/goods/73287855/L
596,OKR 전설적인 벤처투자자가 구글에 전해준 성공 방식,존 도어 박세연 이길상,9.3,경제 경영 경영 경영혁신 경영전략,빌게이츠추천 실리콘밸리 글로벌기업,,http://image.yes24.com/goods/70981291/L
597,취업 끝판왕 옴스에게 배우는 스펙을 뛰어넘는 자소서,옴스,9.8,비즈니스맨 자기계발 CEO 유망직업 수험서 자격증 취업 상식 적성검사 경제 경영 면접,일생의기술,직무 소서 경험 자신,http://image.yes24.com/goods/69338137/L
598,나는 오를 땅만 산다,김종율,9.2,투자 경매 부동산 경제 경영 재테크,돈좀벌어보자 부자되는법 가난이싫어 안정적투자,토지 사례 가치 투자,http://image.yes24.com/goods/67096736/L


In [6]:
book_info_0[book_info_0['rating'] < 8.5]

,title,author,rating,category,tag,keyword,image
13,"아들아, 돈 공부해야 한다",정선용,8.3,투자 경제이야기 재테크 경제 재테크일반 경제 경영 경제상식,부자되는법,소득 스터디 부동산 아들 카페 공부 근로 저자 자본 경제,http://image.yes24.com/goods/98831374/L
27,부의 인문학,브라운스톤,8.4,비즈니스맨 경제 경영 2020년 올해의 책 후보도서 경제사상과 이론 YES24 올해...,돈모으고싶을때 다시보는2020 올해의책 금융문맹탈출 경제와인문의만남 성공하고싶다면...,인문 부의 투자,http://image.yes24.com/goods/79695513/L
85,부자 아빠 가난한 아빠 2,로버트 기요사키 안진환,8.2,원서 VS 번역서 투자 재테크일반 경제 경영 재테크 원서 VS 번역서 [비즈니스],돈좀벌어보자 가난이싫어 부자되는법,,http://image.yes24.com/goods/125459/L
99,한국형 가치투자 전략,최준철 김민국,8.4,주식 투자 재테크 경제 경영 증권,,발굴 투자 기업 사례 분석,http://image.yes24.com/goods/436174/L
144,엘리어트 파동이론,로버트 R. 프렉터 2세,7.6,주식 투자 재테크 재테크일반 경제 경영 증권,시장분석 투자이론,파동 이론 엘리어트 투자 시장 파악,http://image.yes24.com/goods/5247638/L
150,50대 사건으로 보는 돈의 역사,홍춘욱,8.0,세계경제 전망 YES24 올해의 책 경제사 경제 2019년 올해의 책 후보도서 경제...,금융문맹탈출 돈구경시켜드릴게 경제와인문의만남 올해의책 교과서에없는역사 뭐읽지 역사...,역사 세계 사건 흐름 박사 홍춘 배경 인물 금융,http://image.yes24.com/goods/72230588/L
152,주식시장 흐름 읽는 법,우라가미 구니오 박승원,7.1,주식 투자 재테크 경제 경영 증권,,,http://image.yes24.com/goods/39049/L
170,터틀 트레이딩,차영주,8.1,주식 투자 재테크 경제 경영 증권,삼프로TV 주식투자,투자 주식시 전문 수익,http://image.yes24.com/goods/93536906/L
176,파이썬을 이용한 비트코인 자동매매,조대표 유대표,8.0,프로그래밍 언어 투자 주식 IT 모바일 증권 경제 경영 파이썬 재테크,,개발 자동 이용 프로 사용 매매 실시 거래 파이 비트,http://image.yes24.com/goods/98135876/L
213,90년생이 온다 (YES24 스페셜 에디션),임홍택,8.4,경향신문 2019년 올해의 책 선정도서 미래예측 한겨레 세일즈 YES24 올해의 책...,스페셜에디션 걱정은돈으로하세요 올해의책 명절홧병타파 문재인대통령이읽은책 뭐읽지 김...,,http://image.yes24.com/goods/66754547/L


In [12]:
book_info_0[book_info_0['title'].str.contains('시리즈')]

,title,author,rating,category,tag,keyword,image
356,2021 생활세금시리즈,국세청,NaN,경제 경영 정부간행물,,,http://image.yes24.com/goods/101875077/L


In [13]:
book_info_0[book_info_0['title'].str.contains('전집')]

,title,author,rating,category,tag,keyword,image


In [16]:
book_info_0[book_info_0['title'].str.contains('2')]

,title,author,rating,category,tag,keyword,image
3,부자 아빠 가난한 아빠 20주년 특별 기념판,로버트 기요사키 안진환,9.4,경제 경영 투자 재테크 재테크일반,돈모으고싶을때 금융문맹탈출 가난이싫어 부자되는법,아빠 금융 부자 세계 투자 아버 대한 가지 사람 내용,http://image.yes24.com/goods/58774995/L
11,2030 축의 전환,마우로 기옌 우진하,9.2,미래예측 세일즈 마케팅 경제 경영 트렌드,2030 김미경TV 아마존인기도서 미래전망 미래예측,세계 세대,http://image.yes24.com/goods/93888691/L
15,컨버전스 2030,피터 디아만디스 스티븐 코틀러 박영준,9.6,미래예측 한겨레 세일즈 전망 경제전망 마케팅 경제사 경제 미디어 추천 경제 경영 트...,미래전망 미래예측 미래사회 실리콘밸리 2030,,http://image.yes24.com/goods/97247149/L
45,워런 버핏 바이블 2021,이건 최준철 홍영표,10.0,비즈니스맨 주식 투자 CEO 리더십 재테크 간부학 경제 경영 증권,가치투자 주식투자 워런버핏,버핏 바이 투자 해설 워런 최준 한국판 대한,http://image.yes24.com/goods/101684886/L
47,나의 첫 투자 수업 2 투자편,김이안 김정환,9.8,주식 투자 재테크 경제 경영 증권,주린이힘내힘내 주식투자입문 주식투자 주식초보,,http://image.yes24.com/goods/97404824/L
75,트렌드 코리아 2021,김난도 전미영 최지혜 이향은 이준영,9.2,미래예측 세일즈 전망 경제전망 마케팅 경제사 경제 경제 경영 트렌드 각국 경제,소비트렌드 트렌드키워드 비즈니스트렌드,,http://image.yes24.com/goods/93068681/L
85,부자 아빠 가난한 아빠 2,로버트 기요사키 안진환,8.2,원서 VS 번역서 투자 재테크일반 경제 경영 재테크 원서 VS 번역서 [비즈니스],돈좀벌어보자 가난이싫어 부자되는법,,http://image.yes24.com/goods/125459/L
98,2021 상장기업 업종 지도,박찬일,9.6,주식 투자 재테크 경제 경영 증권,주린이힘내힘내 주식투자 상장기업,업종 기업 데이 종목 주식 투자 상장 분류,http://image.yes24.com/goods/96275113/L
103,박 과장은 어떻게 5년 만에 120억을 만들었나,박재진,9.7,투자 경매 부동산 경제 경영 재테크,그래도아파트를사세요 부동산공부 부동산투자 직장인재테크,투자 부동산 직장 아파 생활 시작 자산 대해,http://image.yes24.com/goods/101659217/L
110,규칙 없음 (YES24 스페셜 에디션),리드 헤이스팅스 에린 마이어 이경남,9.3,비즈니스맨 동아일보 2020년 올해의 책 후보도서 YES24 올해의 책 경영혁신 C...,스페셜에디션 다시보는2020 열일하는인사팀 넷플릭스 실리콘밸리 일하고싶은회사 최고...,,http://image.yes24.com/goods/92275597/L


In [17]:
print(book_info_0.duplicated(["title"]))

0      False
1      False
2      False
3      False
4      False
       ...  
595    False
596    False
597    False
598    False
599    False
Length: 600, dtype: bool


In [21]:
book_info_0.drop_duplicates(["title"], keep="first", inplace=True)
book_info_0

,title,author,rating,category,tag,keyword,image
0,매매의 기술,박병창,9.5,주식 투자 재테크 재테크일반 경제 경영 증권,주식투자입문 주식투자,,http://image.yes24.com/goods/101666872/L
1,부의 시나리오,오건영,NaN,경제 경영 투자 재테크 재테크일반,투자트렌드 포트폴리오 삼프로TV,금융시장 투자 경제 오건,http://image.yes24.com/goods/101874075/L
2,라오어의 미국주식 무한매수법,라오어,10.0,주식 투자 재테크 경제 경영 증권,미국주식,,http://image.yes24.com/goods/101875887/L
3,부자 아빠 가난한 아빠 20주년 특별 기념판,로버트 기요사키 안진환,9.4,경제 경영 투자 재테크 재테크일반,돈모으고싶을때 금융문맹탈출 가난이싫어 부자되는법,아빠 금융 부자 세계 투자 아버 대한 가지 사람 내용,http://image.yes24.com/goods/58774995/L
4,나는 당신이 주식 공부를 시작했으면 좋겠습니다,이효석,10.0,주식 투자 재테크 경제 경영 증권,삼프로TV 주식투자입문 주식투자,투자 생존 보이지 주식 저자 미래 전문,http://image.yes24.com/goods/101619905/L
...,...,...,...,...,...,...,...
595,경제용어도감,하나오카 사치코 하마바타케 가노우 송서휘,10.0,경제 경영 경제이야기 경제상식 경제,효율적인공부 상식,경제 이해 하나 직장 일러 용어 문장 개념 모든,http://image.yes24.com/goods/73287855/L
596,OKR 전설적인 벤처투자자가 구글에 전해준 성공 방식,존 도어 박세연 이길상,9.3,경제 경영 경영 경영혁신 경영전략,빌게이츠추천 실리콘밸리 글로벌기업,,http://image.yes24.com/goods/70981291/L
597,취업 끝판왕 옴스에게 배우는 스펙을 뛰어넘는 자소서,옴스,9.8,비즈니스맨 자기계발 CEO 유망직업 수험서 자격증 취업 상식 적성검사 경제 경영 면접,일생의기술,직무 소서 경험 자신,http://image.yes24.com/goods/69338137/L
598,나는 오를 땅만 산다,김종율,9.2,투자 경매 부동산 경제 경영 재테크,돈좀벌어보자 부자되는법 가난이싫어 안정적투자,토지 사례 가치 투자,http://image.yes24.com/goods/67096736/L


In [24]:

book_info_0.drop(book_info_0.loc[book_info_0['author']==''].index, inplace=True)
len(book_info_0)

596

# preprocessing 함수

In [87]:
def preprocessing(df, genre):
    df.drop(df.loc[df['title']==''].index, inplace=True)
    df.drop(df.loc[df['author']==''].index, inplace=True)
    df['rating'] = pd.to_numeric(df['rating'])
    df.drop(df.loc[df['rating']<8.5].index, inplace=True)
    df.drop(df.loc[df['title'].str.contains('시리즈')].index, inplace=True)
    df.drop(df.loc[df['title'].str.contains('전집')].index, inplace=True)
    df.drop_duplicates(['title'], keep="first", inplace=True)
    df.drop(df.loc[df['category'].str.contains('교재')].index, inplace=True)
    if genre == "예술":
        df.drop(df.loc[df['category'].str.contains('악보집')].index, inplace=True)
        df.drop(df.loc[df['category'].str.contains('스케치')].index, inplace=True)
        df.drop(df.loc[df['category'].str.contains('일러스트')].index, inplace=True)
        df.drop(df.loc[df['category'].str.contains('화보집')].index, inplace=True)
        df.drop(df.loc[df['category'].str.contains('아트북')].index, inplace=True)
    elif genre == "소설&시&희곡" or genre == "에세이":
        df.drop(df.loc[df['category'].str.contains('라이트')].index, inplace=True)
        df.drop(df.loc[df['category'].str.contains('만화')].index, inplace=True)
    print(genre + ": " + str(len(df)) + "rows")
    
    df.to_pickle("./crawled_data/pkl/최종/book_info_"+genre+".pkl")

In [89]:
files

['book_info_경제&경영.pkl',
 'book_info_사회&정치.pkl',
 'book_info_소설&시&희곡.pkl',
 'book_info_에세이.pkl',
 'book_info_역사.pkl',
 'book_info_예술.pkl',
 'book_info_인문.pkl',
 'book_info_자기계발.pkl',
 'book_info_자연과학.pkl']

In [90]:
book_info_0 = pd.read_pickle(path+files[0])
book_info_1 = pd.read_pickle(path+files[1])
book_info_2 = pd.read_pickle(path+files[2])
book_info_3 = pd.read_pickle(path+files[3])
book_info_4 = pd.read_pickle(path+files[4])
book_info_5 = pd.read_pickle(path+files[5])
book_info_6 = pd.read_pickle(path+files[6])
book_info_7 = pd.read_pickle(path+files[7])
book_info_8 = pd.read_pickle(path+files[8])

In [91]:
preprocessing(book_info_0, "경제&경영")
preprocessing(book_info_1, "사회&정치")
preprocessing(book_info_2, "소설&시&희곡")
preprocessing(book_info_3, "에세이")
preprocessing(book_info_4, "역사")
preprocessing(book_info_5, "예술")
preprocessing(book_info_6, "인문")
preprocessing(book_info_7, "자기계발")
preprocessing(book_info_8, "자연과학")

경제&경영: 561rows
사회&정치: 544rows
소설&시&희곡: 495rows
에세이: 575rows
역사: 527rows
예술: 239rows
인문: 541rows
자기계발: 548rows
자연과학: 523rows


# 하나의 dataframe으로 모두 합친 후
- 제목 같은 것 하나만 남기기
    - df.drop_duplicates(['title'], keep="first", inplace=True)
- feature 컬럼으로 모으기
- 제목 띄어쓰기 삭제?

In [100]:
book_info_total = pd.concat([book_info_0,book_info_1,book_info_2,book_info_3,book_info_4,book_info_5,book_info_6,book_info_7,book_info_8], ignore_index=True)
book_info_total

,title,author,rating,category,tag,keyword,image
0,매매의 기술,박병창,9.5,주식 투자 재테크 재테크일반 경제 경영 증권,주식투자입문 주식투자,,http://image.yes24.com/goods/101666872/L
1,부의 시나리오,오건영,NaN,경제 경영 투자 재테크 재테크일반,투자트렌드 포트폴리오 삼프로TV,금융시장 투자 경제 오건,http://image.yes24.com/goods/101874075/L
2,라오어의 미국주식 무한매수법,라오어,10.0,주식 투자 재테크 경제 경영 증권,미국주식,,http://image.yes24.com/goods/101875887/L
3,부자 아빠 가난한 아빠 20주년 특별 기념판,로버트 기요사키 안진환,9.4,경제 경영 투자 재테크 재테크일반,돈모으고싶을때 금융문맹탈출 가난이싫어 부자되는법,아빠 금융 부자 세계 투자 아버 대한 가지 사람 내용,http://image.yes24.com/goods/58774995/L
4,나는 당신이 주식 공부를 시작했으면 좋겠습니다,이효석,10.0,주식 투자 재테크 경제 경영 증권,삼프로TV 주식투자입문 주식투자,투자 생존 보이지 주식 저자 미래 전문,http://image.yes24.com/goods/101619905/L
...,...,...,...,...,...,...,...
4548,"아빠, 천체관측 떠나요!",조상호,9.7,천문학 자연과학 별자리와 천체관측,,,http://image.yes24.com/goods/90175509/L
4549,완벽주의자들,사이먼 윈체스터 공경희,9.4,동아일보 주제로 읽는 역사 과학사 과학 역사 중앙일보 세계문화 자연과학 역사와 문화...,완벽주의,역사 사실 정밀 이야 흥미,http://image.yes24.com/goods/90000821/L
4550,인체 의학 도감 베스트 4종 세트,노가미 하루오 마쓰무라 다카히로 야마모토 나오마사 야마구치 슌페이 다케우치 슈지,NaN,의학 재미있는 인체이야기 자연과학 인체 건강에세이 건강기타 건강 취미 약학,,,http://image.yes24.com/goods/89970464/L
4551,세상에서 가장 재미있는 생물학,데이브 웨스너 래리 고닉 김소정,10.0,생물학 생명과학 재미있는 생명이야기 자연과학,,생물학 가장 웨스 세상 시리 하버드 만화 데이브 과학 만점,http://image.yes24.com/goods/89907468/L


In [98]:
book_info_total.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
4548    False
4549     True
4550    False
4551    False
4552    False
Length: 4553, dtype: bool

In [151]:
book_info_total.drop_duplicates(keep="first", inplace=True, ignore_index=True)
book_info_total

,title,author,rating,category,tag,keyword,image,title_concat,feature
0,매매의 기술,박병창,9.5,주식 투자 재테크 재테크일반 경제 경영 증권,주식투자입문 주식투자,,http://image.yes24.com/goods/101666872/L,매매의기술,박병창 주식 투자 재테크 재테크일반 경제 경영 증권 주식투자입문 주식투자
1,부의 시나리오,오건영,NaN,경제 경영 투자 재테크 재테크일반,투자트렌드 포트폴리오 삼프로TV,금융시장 투자 경제 오건,http://image.yes24.com/goods/101874075/L,부의시나리오,오건영 경제 경영 투자 재테크 재테크일반 투자트렌드 포트폴리오 삼프로TV 금융시...
2,라오어의 미국주식 무한매수법,라오어,10.0,주식 투자 재테크 경제 경영 증권,미국주식,,http://image.yes24.com/goods/101875887/L,라오어의미국주식무한매수법,라오어 주식 투자 재테크 경제 경영 증권 미국주식
3,부자 아빠 가난한 아빠 20주년 특별 기념판,로버트 기요사키 안진환,9.4,경제 경영 투자 재테크 재테크일반,돈모으고싶을때 금융문맹탈출 가난이싫어 부자되는법,아빠 금융 부자 세계 투자 아버 대한 가지 사람 내용,http://image.yes24.com/goods/58774995/L,부자아빠가난한아빠20주년특별기념판,로버트 기요사키 안진환 경제 경영 투자 재테크 재테크일반 돈모으고싶을때 금융문맹...
4,나는 당신이 주식 공부를 시작했으면 좋겠습니다,이효석,10.0,주식 투자 재테크 경제 경영 증권,삼프로TV 주식투자입문 주식투자,투자 생존 보이지 주식 저자 미래 전문,http://image.yes24.com/goods/101619905/L,나는당신이주식공부를시작했으면좋겠습니다,이효석 주식 투자 재테크 경제 경영 증권 삼프로TV 주식투자입문 주식투자 투자 ...
...,...,...,...,...,...,...,...,...,...
4239,뇌를 최적화시키는 생활명상 10분,신재한 임운나,NaN,뇌과학 뇌과학 일반 자연과학,,마음 상태 의식 목적 명상,http://image.yes24.com/goods/90584042/L,뇌를최적화시키는생활명상10분,신재한 임운나 뇌과학 뇌과학 일반 자연과학 마음 상태 의식 목적 명상
4240,"아빠, 천체관측 떠나요!",조상호,9.7,천문학 자연과학 별자리와 천체관측,,,http://image.yes24.com/goods/90175509/L,"아빠,천체관측떠나요!",조상호 천문학 자연과학 별자리와 천체관측
4241,인체 의학 도감 베스트 4종 세트,노가미 하루오 마쓰무라 다카히로 야마모토 나오마사 야마구치 슌페이 다케우치 슈지,NaN,의학 재미있는 인체이야기 자연과학 인체 건강에세이 건강기타 건강 취미 약학,,,http://image.yes24.com/goods/89970464/L,인체의학도감베스트4종세트,노가미 하루오 마쓰무라 다카히로 야마모토 나오마사 야마구치 슌페이 다케우치 슈지 ...
4242,세상에서 가장 재미있는 생물학,데이브 웨스너 래리 고닉 김소정,10.0,생물학 생명과학 재미있는 생명이야기 자연과학,,생물학 가장 웨스 세상 시리 하버드 만화 데이브 과학 만점,http://image.yes24.com/goods/89907468/L,세상에서가장재미있는생물학,데이브 웨스너 래리 고닉 김소정 생물학 생명과학 재미있는 생명이야기 자연과학 생...


In [152]:
book_info_total.to_pickle("./crawled_data/pkl/최종/book_info_total.pkl")

### 중간
- 제목 띄어쓰기 없앤 버전을 title_concat 컬럼으로 새로 만들기
- feature 컬럼 만들기

In [153]:
book_info_total['title_concat'] = book_info_total['title'].str.replace(' ','')
book_info_total

,title,author,rating,category,tag,keyword,image,title_concat,feature
0,매매의 기술,박병창,9.5,주식 투자 재테크 재테크일반 경제 경영 증권,주식투자입문 주식투자,,http://image.yes24.com/goods/101666872/L,매매의기술,박병창 주식 투자 재테크 재테크일반 경제 경영 증권 주식투자입문 주식투자
1,부의 시나리오,오건영,NaN,경제 경영 투자 재테크 재테크일반,투자트렌드 포트폴리오 삼프로TV,금융시장 투자 경제 오건,http://image.yes24.com/goods/101874075/L,부의시나리오,오건영 경제 경영 투자 재테크 재테크일반 투자트렌드 포트폴리오 삼프로TV 금융시...
2,라오어의 미국주식 무한매수법,라오어,10.0,주식 투자 재테크 경제 경영 증권,미국주식,,http://image.yes24.com/goods/101875887/L,라오어의미국주식무한매수법,라오어 주식 투자 재테크 경제 경영 증권 미국주식
3,부자 아빠 가난한 아빠 20주년 특별 기념판,로버트 기요사키 안진환,9.4,경제 경영 투자 재테크 재테크일반,돈모으고싶을때 금융문맹탈출 가난이싫어 부자되는법,아빠 금융 부자 세계 투자 아버 대한 가지 사람 내용,http://image.yes24.com/goods/58774995/L,부자아빠가난한아빠20주년특별기념판,로버트 기요사키 안진환 경제 경영 투자 재테크 재테크일반 돈모으고싶을때 금융문맹...
4,나는 당신이 주식 공부를 시작했으면 좋겠습니다,이효석,10.0,주식 투자 재테크 경제 경영 증권,삼프로TV 주식투자입문 주식투자,투자 생존 보이지 주식 저자 미래 전문,http://image.yes24.com/goods/101619905/L,나는당신이주식공부를시작했으면좋겠습니다,이효석 주식 투자 재테크 경제 경영 증권 삼프로TV 주식투자입문 주식투자 투자 ...
...,...,...,...,...,...,...,...,...,...
4239,뇌를 최적화시키는 생활명상 10분,신재한 임운나,NaN,뇌과학 뇌과학 일반 자연과학,,마음 상태 의식 목적 명상,http://image.yes24.com/goods/90584042/L,뇌를최적화시키는생활명상10분,신재한 임운나 뇌과학 뇌과학 일반 자연과학 마음 상태 의식 목적 명상
4240,"아빠, 천체관측 떠나요!",조상호,9.7,천문학 자연과학 별자리와 천체관측,,,http://image.yes24.com/goods/90175509/L,"아빠,천체관측떠나요!",조상호 천문학 자연과학 별자리와 천체관측
4241,인체 의학 도감 베스트 4종 세트,노가미 하루오 마쓰무라 다카히로 야마모토 나오마사 야마구치 슌페이 다케우치 슈지,NaN,의학 재미있는 인체이야기 자연과학 인체 건강에세이 건강기타 건강 취미 약학,,,http://image.yes24.com/goods/89970464/L,인체의학도감베스트4종세트,노가미 하루오 마쓰무라 다카히로 야마모토 나오마사 야마구치 슌페이 다케우치 슈지 ...
4242,세상에서 가장 재미있는 생물학,데이브 웨스너 래리 고닉 김소정,10.0,생물학 생명과학 재미있는 생명이야기 자연과학,,생물학 가장 웨스 세상 시리 하버드 만화 데이브 과학 만점,http://image.yes24.com/goods/89907468/L,세상에서가장재미있는생물학,데이브 웨스너 래리 고닉 김소정 생물학 생명과학 재미있는 생명이야기 자연과학 생...


In [154]:
book_info_total['feature'] = book_info_total['author'] + ' ' + book_info_total['category'] + ' ' + book_info_total['tag'] + ' ' + book_info_total['keyword']
book_info_total[['feature']]

,feature
0,박병창 주식 투자 재테크 재테크일반 경제 경영 증권 주식투자입문 주식투자
1,오건영 경제 경영 투자 재테크 재테크일반 투자트렌드 포트폴리오 삼프로TV 금융시...
2,라오어 주식 투자 재테크 경제 경영 증권 미국주식
3,로버트 기요사키 안진환 경제 경영 투자 재테크 재테크일반 돈모으고싶을때 금융문맹...
4,이효석 주식 투자 재테크 경제 경영 증권 삼프로TV 주식투자입문 주식투자 투자 ...
...,...
4239,신재한 임운나 뇌과학 뇌과학 일반 자연과학 마음 상태 의식 목적 명상
4240,조상호 천문학 자연과학 별자리와 천체관측
4241,노가미 하루오 마쓰무라 다카히로 야마모토 나오마사 야마구치 슌페이 다케우치 슈지 ...
4242,데이브 웨스너 래리 고닉 김소정 생물학 생명과학 재미있는 생명이야기 자연과학 생...


# tf-idf vectorizer

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.metrics.pairwise import linear_kernel

In [4]:
book_info_total = pd.read_pickle('./book_info_total.pkl')
book_info_total

,title,author,rating,category,tag,keyword,image,title_concat,feature
0,매매의 기술,박병창,9.5,주식 투자 재테크 재테크일반 경제 경영 증권,주식투자입문 주식투자,,http://image.yes24.com/goods/101666872/L,매매의기술,박병창 주식 투자 재테크 재테크일반 경제 경영 증권 주식투자입문 주식투자
1,부의 시나리오,오건영,NaN,경제 경영 투자 재테크 재테크일반,투자트렌드 포트폴리오 삼프로TV,금융시장 투자 경제 오건,http://image.yes24.com/goods/101874075/L,부의시나리오,오건영 경제 경영 투자 재테크 재테크일반 투자트렌드 포트폴리오 삼프로TV 금융시...
2,라오어의 미국주식 무한매수법,라오어,10.0,주식 투자 재테크 경제 경영 증권,미국주식,,http://image.yes24.com/goods/101875887/L,라오어의미국주식무한매수법,라오어 주식 투자 재테크 경제 경영 증권 미국주식
3,부자 아빠 가난한 아빠 20주년 특별 기념판,로버트 기요사키 안진환,9.4,경제 경영 투자 재테크 재테크일반,돈모으고싶을때 금융문맹탈출 가난이싫어 부자되는법,아빠 금융 부자 세계 투자 아버 대한 가지 사람 내용,http://image.yes24.com/goods/58774995/L,부자아빠가난한아빠20주년특별기념판,로버트 기요사키 안진환 경제 경영 투자 재테크 재테크일반 돈모으고싶을때 금융문맹...
4,나는 당신이 주식 공부를 시작했으면 좋겠습니다,이효석,10.0,주식 투자 재테크 경제 경영 증권,삼프로TV 주식투자입문 주식투자,투자 생존 보이지 주식 저자 미래 전문,http://image.yes24.com/goods/101619905/L,나는당신이주식공부를시작했으면좋겠습니다,이효석 주식 투자 재테크 경제 경영 증권 삼프로TV 주식투자입문 주식투자 투자 ...
...,...,...,...,...,...,...,...,...,...
4239,뇌를 최적화시키는 생활명상 10분,신재한 임운나,NaN,뇌과학 뇌과학 일반 자연과학,,마음 상태 의식 목적 명상,http://image.yes24.com/goods/90584042/L,뇌를최적화시키는생활명상10분,신재한 임운나 뇌과학 뇌과학 일반 자연과학 마음 상태 의식 목적 명상
4240,"아빠, 천체관측 떠나요!",조상호,9.7,천문학 자연과학 별자리와 천체관측,,,http://image.yes24.com/goods/90175509/L,"아빠,천체관측떠나요!",조상호 천문학 자연과학 별자리와 천체관측
4241,인체 의학 도감 베스트 4종 세트,노가미 하루오 마쓰무라 다카히로 야마모토 나오마사 야마구치 슌페이 다케우치 슈지,NaN,의학 재미있는 인체이야기 자연과학 인체 건강에세이 건강기타 건강 취미 약학,,,http://image.yes24.com/goods/89970464/L,인체의학도감베스트4종세트,노가미 하루오 마쓰무라 다카히로 야마모토 나오마사 야마구치 슌페이 다케우치 슈지 ...
4242,세상에서 가장 재미있는 생물학,데이브 웨스너 래리 고닉 김소정,10.0,생물학 생명과학 재미있는 생명이야기 자연과학,,생물학 가장 웨스 세상 시리 하버드 만화 데이브 과학 만점,http://image.yes24.com/goods/89907468/L,세상에서가장재미있는생물학,데이브 웨스너 래리 고닉 김소정 생물학 생명과학 재미있는 생명이야기 자연과학 생...


In [5]:
# 사용자가 검색어로 넣는 책 이름을 띄어쓰기를 해도, 안해도 index 반환 가능하도록 구현함
def get_book_idx(book_title):
    book_idx = book_info_total.index[(book_info_total['title_concat']==book_title) | (book_info_total['title']==book_title)].to_list()
    if book_idx:
        return book_idx[0]
    else:
        return None

def get_book_title(book_idx):
    if book_idx in book_info_total.index.values:
        return book_info_total.loc[book_idx]['title']
    else:
        return None

In [6]:
print(get_book_title(get_book_idx('뇌를 최적화시키는 생활명상 10분')))
print(get_book_title(get_book_idx('뇌를최적화시키는생활명상10분')))

뇌를 최적화시키는 생활명상 10분
뇌를 최적화시키는 생활명상 10분


In [7]:
# min_df를 1로 설정해줌으로써 한번이라도 노출된 정보도 다 고려함
# ngram_range: n_gram 범위 지정 연속으로 나오는 단어들의 순서도 고려함
tf = TfidfVectorizer(min_df=1, ngram_range=(1,5))
tfidf_matrix = tf.fit_transform(book_info_total['feature'])
tfidf_matrix

<4244x165244 sparse matrix of type '<class 'numpy.float64'>'
	with 286929 stored elements in Compressed Sparse Row format>

In [8]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.07425905, 0.14541622, ..., 0.        , 0.        ,
        0.        ],
       [0.07425905, 1.        , 0.04862108, ..., 0.        , 0.        ,
        0.        ],
       [0.14541622, 0.04862108, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.00415701,
        0.00476445],
       [0.        , 0.        , 0.        , ..., 0.00415701, 1.        ,
        0.00457294],
       [0.        , 0.        , 0.        , ..., 0.00476445, 0.00457294,
        1.        ]])

In [9]:
def similar_books_by_title(title, n=10):
    # 해당 제목의 책의 index를 구한다.
    book_idx = get_book_idx(title)
    if book_idx is None:
        print("해당 도서는 데이터셋에 존재하지 않습니다. 다른 도서나 키워드로 다시 검색해주세요.")
    else:
        print(book_idx)
        # 모든 책에 대해서 해당 책과의 cosine similarity를 구한다.
        book_similarities = list(enumerate(cosine_sim[book_idx]))
        # cosine similarity에 따라 책들을 정렬한다.
        book_similarities = sorted(book_similarities, key=lambda x:x[1], reverse=True)
        most_similar_books = list(map(lambda x: (get_book_title(x[0]), round(x[1],2)), book_similarities[:n+1]))
        most_similar_books = list(filter(lambda x: title != x[0], most_similar_books))
        
        print(most_similar_books)

In [10]:
similar_books_by_title('뇌를최적화시키는생활명상10분')

4239
[('뇌를 최적화시키는 생활명상 10분', 1.0), ('박문호 박사의 뇌과학 공부', 0.14), ('그림으로 읽는 뇌과학의 모든 것', 0.14), ('내 머릿속에선 무슨 일이 벌어지고 있을까', 0.14), ('지능의 탄생', 0.12), ('당신의 뇌, 미래의 뇌', 0.11), ('NEWTON HIGHLIGHT 뉴턴 하이라이트 128 기본부터 알아보는 심리학', 0.1), ('더 브레인', 0.1), ('마음의 오류들 ', 0.09), ('사이코패스 뇌과학자', 0.08), ('슈퍼인텔리전스', 0.08)]


In [11]:
similar_books_by_title('나는 당신이 주식 공부를 시작했으면 좋겠습니다')

4
[('투자노트 Investing notes', 0.24), ('현명한 당신의 주식 투자 교과서', 0.23), ('시간여행TV의 주식투자전략', 0.23), ('시간여행TV의 테마주 백과사전', 0.23), ('전인구의 주식투자 일주일 만에 뽀개기', 0.23), ('좋은 주식 나쁜 주식', 0.22), ('돈 버는 주식투자 비법', 0.22), ('하루 만에 수익 내는 데이트레이딩 3대 타법', 0.22), ('투자의 지도', 0.22), ('가치투자 처음공부', 0.22)]
